In [ ]:
# installing kaggle library
!pip install kaggle

Upload Kaggle.json file which contains kaggle credential.

In [ ]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing the Twitter dataset using api

In [ ]:
# API to fetch the dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.47GB/s]


In [ ]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import re # regular expression : used for pattern matching
from nltk.corpus import stopwords # nltk : natural language toolkit -> used with text feature
from nltk.stem.porter import PorterStemmer # used to reduce the word size to root word
from sklearn.feature_extraction.text import TfidfVectorizer  # used to convert textual data into numerical daa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Model does not require this words so we remove such words from the dataset.

Data processing :

In [ ]:
# loading the data from csv file to pandas dataframe
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
# Checking the no of rows and columns
data.shape

(1599999, 6)

In [ ]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
# naming the columns
columns = ['target','ids','date','flag','user','text']
data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=columns,encoding='ISO-8859-1')
data.sample(5)

,target,ids,date,flag,user,text
1085984,4,1969254900,Fri May 29 23:28:19 PDT 2009,NO_QUERY,xxlindi,"@markhoppus on myspace, leave a comment please..."
821646,4,1554107443,Sat Apr 18 16:05:17 PDT 2009,NO_QUERY,RyanFarley,"@bluapp oh yeah, and minimizing to tray would ..."
698727,0,2254223009,Sat Jun 20 09:25:05 PDT 2009,NO_QUERY,mcampion1,"Tiling the kitchen, Fixing a water pipe in the..."
145033,0,1882112313,Fri May 22 06:15:20 PDT 2009,NO_QUERY,Janapan,My dating buddy's not here
1198733,4,1985197806,Sun May 31 16:37:43 PDT 2009,NO_QUERY,vanillatea,http://bit.ly/V5r8a today's stop motion inspir...


In [ ]:
data.shape

(1600000, 6)

In [ ]:
# chekcking the missing values :
data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
# checking the distribution of target column
data['target'].value_counts()

,count
target,
0,800000
4,800000


converting the target "4" to "1" :

In [ ]:
data.replace({'target':{4,1}},inplace=True)

In [ ]:
data['target'].value_counts()

,count
target,
0,800000
1,800000


0--> Negative
1--> Positive

**Stemming:** technique that reduces a word to its root or base form by cutting off prefixes and suffixes. Its purpose is to simplify text for analysis by converting different variations of a word, like "running," "runs," and "ran," to their common stem, "run"

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) # it will remove letters which are not alphabet.
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split() # contents into list of words
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content) # list of words into content
  return stemmed_content

In [ ]:
data['stemmed_content'] = data['text'].apply(stemming)

In [ ]:
print(data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
# separating the input and label(output)
X= data['stemmed_content']
Y= data['target']

Splitting the data to training data and test data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


Converting textual to numerical data.

In [ ]:
vectorizer = TfidfVectorizer()
# It actually assigns some importance value to each word which depends on the no of repeation of that word in a content and then it
# corresponds the importance value to target value (0 or 1) or (Positive or negative post)
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

Trainig the model using Logistic Regression

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy on the training data

In [ ]:
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)
training_data_accuracy

0.79871953125

Accuracy on the testing data

In [ ]:
x_test_prediction = model.predict(x_test)
testing_data_accuracy = accuracy_score(x_test_prediction,y_test)
testing_data_accuracy

0.77668125

Model Accuracy : 77.8%

Saving the trained model

In [ ]:
import pickle

In [ ]:
filename= 'trained_model.sav'
pickle.dump(model,open(filename,'wb')) # writing in binary format

Using the saved model for future prediction

In [ ]:
# loading the saved model
filename = '/content/trained_model.sav'
loaded_model = pickle.load(open(filename,'rb'))

In [ ]:
x_test = pd.DataFrame(x_test)
y_test = pd.DataFrame(y_test)

In [ ]:
X_new = x_test.iloc[20]
print(y_test.iloc[20])

0    <Compressed Sparse Row sparse matrix of dtype ...
Name: 20, dtype: object
target    1
Name: 1371037, dtype: int64


In [ ]:
prediction = loaded_model.predict(X_new.iloc[0])
if(prediction[0]==1):
  print("Positive post")
elif(prediction[0]==0):
  print("Negative post")

Positive post
